In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from crewai import LLM

llm = LLM(
  model = "gemini/gemini-2.0-flash",
  temperature = 0.1,
)

In [2]:
original_email = """
looping in Priya. TAS and PRX updates are in deck. ETA for SDS integration is Friday.
Let's sync up tomorrow if SYNCBOT allows 😄. ping me if any blockers.
"""

from crewai.tools import BaseTool

class ReplaceJargonsTool(BaseTool):
  name: str = "Replace Jargons"
  description: str = "Replaces jargons and abbreviations in the email with their full forms."

  def _run(self, email: str) -> str:
    replacements = {
      "PRX": "Project Phoenix (internal AI revamp project)",
      "TAS": "Technical Architecture Stack",
      "DBX": "Client Database Syncer",
      "SDS": "Smart Data Syncer",
      "SYNCBOT": "Internal Standup Assistant Bot",
      "WIP": "in progess",
      "POC": "proof of concept",
      "ping": "reach out"
    }
    suggestions = []
    email_lower = email.lower()
    for jargon, replacement in replacements.items():
      if jargon.lower() in email_lower:
        suggestions.append(f'Consider replacing "{jargon}" with "{replacement}"')
        
    return "\n".join(suggestions) if suggestions else "No jargons or internal abbreviation detected."
  
jt = ReplaceJargonsTool()
jt.run(original_email)

Using Tool: Replace Jargons


'Consider replacing "PRX" with "Project Phoenix (internal AI revamp project)"\nConsider replacing "TAS" with "Technical Architecture Stack"\nConsider replacing "SDS" with "Smart Data Syncer"\nConsider replacing "SYNCBOT" with "Internal Standup Assistant Bot"\nConsider replacing "ping" with "reach out"'

In [ ]:
from crewai import Agent, Task, Crew

email_assistant = Agent(
    role="Email Assistant Agent",
    goal="Improve emails and make them sound professional and clear",
    backstory="A highly experienced communication expert skilled in professional email writing",
    verbose=True,
    llm=llm
)

email_task = Task(
    description=f"""Take the following rough email and rewrite it into a professional and polished version.
    Expand abbreviations:
    '''{original_email}'''""",
    agent=email_assistant,
    expected_output="A professional written email with proper formatting and content.",
)

crew = Crew(
    agents=[email_assistant],
    tasks=[email_task],
    verbose=True
)

result = crew.kickoff()
print(result)